In [1]:
# sm = сырая таблица
# sm1 = слайс по нужным столбцам и строкам + столбец с месяцем
# sm2 = анпивот по тарифам в плоскую таблицу

In [1]:
import pandas as pd
from pyxlsb import open_workbook
import io
import csv
from datetime import datetime
import os
from os.path import join
import re
import teradatasql
import numpy as np

class SkipRowsError(Exception):
    pass

class ReportMonthError(Exception):
    pass

In [17]:
def process_file(filename, union_df):
    
    with open_workbook(filename) as wb:
        output = io.StringIO()
        writer = csv.writer(output, quoting=csv.QUOTE_NONNUMERIC)
        with wb.get_sheet('Channels_MIX') as sheet:
            for row in sheet.rows():
                csv_line = [r.v for r in row]
                writer.writerow(csv_line)
        output.seek(0)
    csvreader = csv.reader(output)

    # In[111]:

    output.seek(0)
    row1 = next(csvreader)
    try:
        ordinal_days = int(float(row1[0]))
    except:
        raise ReportMonthError
    report_month_dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + ordinal_days - 2).date()
    print(f'Отчетный месяц: {report_month_dt}')


    # In[113]:

    output.seek(0)
    for row_number, row in enumerate(csvreader):
        if row[0] == 'Регион':
            col_row = row_number
            break
    else:
        raise SkipRowsError

    # In[117]:

    col_indexes = [i for i in range(4)]  # столбцы
    for index, cell_value in enumerate(row):
        try:
            if 'план' in cell_value: col_indexes.append(index)
        except:
            continue
    print(f'Старые столбцы: {[row[index] for index in col_indexes]}')


    # In[117]:

    output.seek(0)
    sm = pd.read_csv(output, skiprows=col_row)
    sm1 = sm.iloc[:, col_indexes].dropna()

    # In[118]:

    col_names = {
        'Регион': 'region',
        'Channels': 'channel',
        'МР': 'macroregion',
        'Кластер': 'cluster',
        'Мой безлимит': 'Безлимит',
        'Мой онлайн +': 'Мой онлайн+',
        '*любое старое название': '*любое новое название',

    }
    sm1 = sm1.rename(columns=lambda x: re.sub(r'(.+)\s*план', r'\1', x).strip()).rename(columns=col_names)
    sm1 = sm1.rename(columns=lambda x: re.sub(r'(.+)\s*план', r'\1', x).strip()).rename(columns=col_names)
    sm1.insert(0, 'report_month', report_month_dt)
    if 'Везде онлайн' not in sm1.columns: sm1['Везде онлайн'] = 0
    if 'Везде онлайн+' not in sm1.columns: sm1['Везде онлайн+'] = 0
    print(f'Новые столбцы: {[sm1.columns.to_list()]}')
    # In[122]:

    fixed_cols_count = 5
    sm2 = sm1.melt(id_vars=sm1.columns[:fixed_cols_count].to_list(),
                   value_vars=sm1.columns[fixed_cols_count:].to_list(),
                   var_name='tariff',
                   value_name='subs_count')
    sm2 = sm2.loc[(sm2.channel != 'Total'), :].dropna()

    # In[123]:

    sm2pivot = sm2.groupby(['tariff', 'channel']).sum().reset_index()
    sm2pivot.pivot(index='tariff', columns='channel', values='subs_count')
    

    # In[132]:

    union_df = pd.concat([union_df, sm2]).drop_duplicates()
    return union_df

In [18]:
dirname = r'\\corp.tele2.ru\\plm_cluster\\All\\_Плановый Sales mix\\for Tableau\\'
union_df = pd.DataFrame()

In [19]:
for filename in os.listdir(dirname):
    print(filename, '...')

    # обработать filename, засунуть его в union_df и вернуть обогащенный union_df
    union_df = process_file(join(dirname,filename), union_df)

    print(f'Промежуточный размер датафрейма: {union_df.shape}\n')

Расчет целевого Sales mix_август.xlsb ...
Отчетный месяц: 2020-08-01
Старые столбцы: ['Регион', 'Channels', 'МР', 'Кластер', 'Классический  план', 'Мой Tele2  план', 'Мой разговор  план', 'Мой онлайн  план', 'Мой онлайн +  план', 'Безлимит  план', 'Везде онлайн  план', 'Other  план']
Новые столбцы: [['report_month', 'region', 'channel', 'macroregion', 'cluster', 'Классический', 'Мой Tele2', 'Мой разговор', 'Мой онлайн', 'Мой онлайн+', 'Безлимит', 'Везде онлайн', 'Other', 'Везде онлайн+']]
Промежуточный размер датафрейма: (3906, 7)

Расчет целевого Sales mix_апрель.xlsb ...
Отчетный месяц: 2020-04-01
Старые столбцы: ['Регион', 'Channels', 'МР', 'Кластер', 'Классический план', 'Мой Tele2 план', 'Мой разговор план', 'Мой онлайн план', 'Мой онлайн + план', 'Мой безлимит план', 'Other план']
Новые столбцы: [['report_month', 'region', 'channel', 'macroregion', 'cluster', 'Классический', 'Мой Tele2', 'Мой разговор', 'Мой онлайн', 'Мой онлайн+', 'Безлимит', 'Other', 'Везде онлайн', 'Везде онла

In [49]:
new = union_df.groupby(['tariff','report_month'])['subs_count'].sum().reset_index()
new.pivot(index='tariff', columns='report_month')['subs_count']

report_month,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01
tariff,,,,,,,,,
Other,66066.9,70717.3,96858.9,87171.4,69484.7,22430,31237,41210,60040
Безлимит,644.131846153846125.8593076923076963.472846153...,40561,581.1184041393251113.576212785561653.732945796...,38246,24458.4,32054,34511,39630,38024
Везде онлайн,0,0,0,0,0,65626,91434,109165,93397
Везде онлайн+,0,0,0,0,0,0,0,0,0
Классический,133922,141232,157592,127092,93140.6,89780,108316,123902,115201
Мой Tele2,124573,121458,132085,115731,61189.4,70071,75783,90657,85176
Мой онлайн,371254,364358,373677,358712,227963,275700,341140,426628,439498
Мой онлайн+,188527,198182,214159,205358,79418.3,109252,149307,189460,192430
Мой разговор,221486,223780,241800,225413,170673,195220,241710,293927,291331


In [22]:
with teradatasql.connect() as session:
    query = '''
        sel
            case when name_report in
                ('Классический','Мой разговор','Везде онлайн','Мой Tele2','Мой онлайн','Мой онлайн+','Безлимит')
            then name_report
            else 'Other'
            end as name_report,
            min(tp_id)
        from PRD2_DIC_V.PRICE_PLAN
        group by 1;

    '''
    price_plan_df = pd.read_sql(query, session)
price_plan_df

,name_report,TP_ID
0,Классический,10168.0
1,Мой разговор,10623.0
2,Везде онлайн,20015909.0
3,Безлимит,15960.0
4,Мой онлайн,10628.0
5,Мой Tele2,10624.0
6,Мой онлайн+,10630.0
7,Other,-912.0


In [23]:
union_df_with_tp_id = pd.merge(union_df, price_plan_df, how='inner', left_on='tariff', right_on='name_report')
union_df_with_tp_id.head()

,report_month,region,channel,macroregion,cluster,tariff,subs_count,name_report,TP_ID
0,2020-08-01,Владимир,MB,Центр,Challenger Elite,Классический,996,Классический,10168.0
1,2020-08-01,Владимир,LA,Центр,Challenger Elite,Классический,430,Классический,10168.0
2,2020-08-01,Владимир,LS,Центр,Challenger Elite,Классический,124,Классический,10168.0
3,2020-08-01,Владимир,Stands,Центр,Challenger Elite,Классический,70,Классический,10168.0
4,2020-08-01,Владимир,FA,Центр,Challenger Elite,Классический,300,Классический,10168.0


In [24]:
union_df_with_tp_id['param_1'] = 'SM Plan'
union_df_with_tp_id.head()

,report_month,region,channel,macroregion,cluster,tariff,subs_count,name_report,TP_ID,param_1
0,2020-08-01,Владимир,MB,Центр,Challenger Elite,Классический,996,Классический,10168.0,SM Plan
1,2020-08-01,Владимир,LA,Центр,Challenger Elite,Классический,430,Классический,10168.0,SM Plan
2,2020-08-01,Владимир,LS,Центр,Challenger Elite,Классический,124,Классический,10168.0,SM Plan
3,2020-08-01,Владимир,Stands,Центр,Challenger Elite,Классический,70,Классический,10168.0,SM Plan
4,2020-08-01,Владимир,FA,Центр,Challenger Elite,Классический,300,Классический,10168.0,SM Plan


In [25]:
with teradatasql.connect() as session:
    query = '''
        sel DISTINCT
                b.branch_id,
                r.region_name
        from PRD2_DIC_V.BRANCH b
        inner join PRD2_DIC_V.REGION r
            on b.region_id=r.region_id
        where product_cluster_name is not null
            and branch_id is not null
            and b.branch_name not like '%CDMA%'
            and b.branch_name not like '%MVNO%'
            and b.branch_name not like '%LTE450%'
            and b.product_cluster_name<>'Deferred';

    '''
    region_df = pd.read_sql(query, session)
region_df.head()

,BRANCH_ID,REGION_NAME
0,2.0,Кемерово
1,3.0,Ростов на Дону
2,8.0,С.Петербург
3,9.0,Омск
4,10.0,Ижевск


In [26]:
region_dict = {
    'Краснодар':'Краснодар и Адыгея'   , 
    'Салехард':'ЯНАО'    ,    
    'Абакан':'Хакасия'      ,      
    'Чебоксары':'Чувашия'      ,
    'Ростов-на-Дону':'Ростов на Дону' ,
    'Санкт-Петербург':'С.Петербург'  ,
    'Ханты-Мансийск':'ХМАО' ,
    'Нижний Новгород':'Н.Новгород'   , 
    'Великий Новгород':'В.Новгород'  ,
    'Саранск':'Мордовия'     ,
    'Йошкар-Ола':'Марий Эл'    ,
    'Улан-Удэ':'Бурятия'   ,  
}
union_df_with_tp_id.region = union_df_with_tp_id.region.replace(region_dict)
union_df_with_branch_id = pd.merge(union_df_with_tp_id, region_df,
                               how='left', left_on='region', right_on='REGION_NAME')
union_df_with_branch_id.head()

,report_month,region,channel,macroregion,cluster,tariff,subs_count,name_report,TP_ID,param_1,BRANCH_ID,REGION_NAME
0,2020-08-01,Владимир,MB,Центр,Challenger Elite,Классический,996,Классический,10168.0,SM Plan,28.0,Владимир
1,2020-08-01,Владимир,LA,Центр,Challenger Elite,Классический,430,Классический,10168.0,SM Plan,28.0,Владимир
2,2020-08-01,Владимир,LS,Центр,Challenger Elite,Классический,124,Классический,10168.0,SM Plan,28.0,Владимир
3,2020-08-01,Владимир,Stands,Центр,Challenger Elite,Классический,70,Классический,10168.0,SM Plan,28.0,Владимир
4,2020-08-01,Владимир,FA,Центр,Challenger Elite,Классический,300,Классический,10168.0,SM Plan,28.0,Владимир


In [27]:
na_branch = union_df_with_branch_id[union_df_with_branch_id.BRANCH_ID.isna()]
na_branch.region.value_counts()
#если пусто, значит все регионы нашлись - все ок

Series([], Name: region, dtype: int64)

In [28]:
union_df_with_branch_id = union_df_with_branch_id[union_df_with_branch_id.subs_count != '0x7']
union_df_with_branch_id.subs_count = union_df_with_branch_id.subs_count.astype(float)
union_df_with_branch_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37800 entries, 0 to 37813
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_month  37800 non-null  object 
 1   region        37800 non-null  object 
 2   channel       37800 non-null  object 
 3   macroregion   37800 non-null  object 
 4   cluster       37800 non-null  object 
 5   tariff        37800 non-null  object 
 6   subs_count    37800 non-null  float64
 7   name_report   37800 non-null  object 
 8   TP_ID         37800 non-null  float64
 9   param_1       37800 non-null  object 
 10  BRANCH_ID     37800 non-null  float64
 11  REGION_NAME   37800 non-null  object 
dtypes: float64(3), object(9)
memory usage: 3.7+ MB


In [29]:
# чистим product_parameters_test_for_sap и заполняем данными
cols = ['report_month', 'param_1', 'channel',
        'NULL', 'NULL1', 'BRANCH_ID', 'NULL6',
        'TP_ID', 'NULL3', 'NULL7', 'NULL4', 'subs_count']
df = union_df_with_branch_id.reindex(columns = cols,
                                     fill_value = np.NaN).where((pd.notnull(union_df_with_branch_id)), None)
with teradatasql.connect(host='td2800.corp.tele2.ru', user='PLOTNIKOV_II',password='Anna062020') as con:
    with con.cursor() as cur:
 
        print('deleting from uat_product.product_parameters...')
        cur.execute('''
            delete from UAT_PRODUCT.PRODUCT_PARAMETERS
                where param_1 = 'SM Plan'
        ''')
        print(f'{cur.rowcount} rows deleted from uat_product.product_parameters')
        
        batchsize = 100000
        print('inserting into UAT_PRODUCT.PRODUCT_PARAMETERS...')
        for num in range(0, len(df), batchsize):
            cur.executemany(f'''
             INSERT into UAT_PRODUCT.PRODUCT_PARAMETERS ({','.join('?'*len(df.columns))})
            ''',
                [tuple(row) for row in df.iloc[num:num+batchsize,:].itertuples(index=False)]
                )
        print(f'{len(df)} rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS in total.')

deleting from uat_product.product_parameters...
32690 rows deleted from uat_product.product_parameters
inserting into UAT_PRODUCT.PRODUCT_PARAMETERS...
37800 rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS in total.
